In [10]:
# Code for Newton method
import numpy as np
import timeit

np.random.seed(10)  # for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N, 1)  # random noise

# Define the Newton's method function
def newton_method(A, y, lambda_reg, tol=1e-6, max_iter=100):
    d = A.shape[1]
    x = np.zeros((d, 1))
    I = np.identity(d)
    
    for _ in range(max_iter):
        grad = A.T @ (A @ x - y) + lambda_reg * x
        hessian = A.T @ A + lambda_reg * I
        delta_x = np.linalg.solve(hessian, -grad)
        x = x + delta_x
        
        if np.linalg.norm(delta_x) < tol:
            break
    
    return x

# For each value of dimension in the ds array, we will check the behavior of Newton's method
for i in range(np.size(ds)):
    d = ds[i]
    A = np.random.randn(N, d)
    
    # Normalize the columns
    for j in range(A.shape[1]):
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
    
    xorig = np.ones((d, 1))
    y = np.dot(A, xorig) + eps
    
    start = timeit.default_timer()
    
    # Call Newton method with A, y, lambda and obtain the optimal solution x_opt
    x_opt = newton_method(A, y, lambda_reg)
    
    newtontime = timeit.default_timer() - start  # time is in seconds
    
    # Print the total time and the L2 norm difference || x_opt - xorig || for Newton method
    l2_norm_diff = np.linalg.norm(x_opt - xorig)
    print(f"Dimension: {d}, Time: {newtontime:.4f} seconds, L2 Norm Difference: {l2_norm_diff:.4f}")

Dimension: 1000, Time: 0.1599 seconds, L2 Norm Difference: 28.7546
Dimension: 5000, Time: 10.5770 seconds, L2 Norm Difference: 69.3254
Dimension: 10000, Time: 27.3749 seconds, L2 Norm Difference: 98.8945
Dimension: 20000, Time: 415.3999 seconds, L2 Norm Difference: 140.5886
Dimension: 25000, Time: 623.5029 seconds, L2 Norm Difference: 157.5001


MemoryError: Unable to allocate 18.6 GiB for an array with shape (50000, 50000) and data type float64

In [11]:
import numpy as np
import timeit

np.random.seed(1000)  # for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N, 1)  # random noise

# Define the BFGS method function
def bfgs_method(A, y, lambda_reg, tol=1e-6, max_iter=100):
    d = A.shape[1]
    x = np.zeros((d, 1))
    I = np.identity(d)
    H = I
    
    for _ in range(max_iter):
        grad = A.T @ (A @ x - y) + lambda_reg * x
        p = -H @ grad
        alpha = 1e-3  # You may need to implement a proper line search for alpha

        s = alpha * p
        x_new = x + s

        yk = A.T @ (A @ x_new - y) + lambda_reg * x_new - grad
        skT_yk = s.T @ yk

        if np.abs(skT_yk) < tol:
            break

        H = H + (skT_yk + yk.T @ H @ yk) * (s @ s.T) / (skT_yk ** 2) - (H @ yk @ s.T + s @ yk.T @ H) / skT_yk
        x = x_new

        if np.linalg.norm(grad) < tol:
            break
    
    return x

# For each value of dimension in the ds array, we will check the behavior of both methods
for i in range(np.size(ds)):
    d = ds[i]
    A = np.random.randn(N, d)
    
    # Normalize the columns
    for j in range(A.shape[1]):
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
    
    xorig = np.ones((d, 1))
    y = np.dot(A, xorig) + eps
    
    
    # BFGS method
    start = timeit.default_timer()
    x_opt_bfgs = bfgs_method(A, y, lambda_reg)
    bfgstime = timeit.default_timer() - start  # time is in seconds
    
    # Calculate and print results for BFGS method
    norm_diff_bfgs = np.linalg.norm(x_opt_bfgs - xorig)
    residual_bfgs = np.linalg.norm(A @ x_opt_bfgs - y) ** 2
    print(f"BFGS Method - Dimension: {d}, Time: {bfgstime:.4f} seconds, Residual: {residual_bfgs:.4f}, L2 Norm Difference: {norm_diff_bfgs:.4f}")

BFGS Method - Dimension: 1000, Time: 8.1804 seconds, Residual: 837.2461, L2 Norm Difference: 30.8116
BFGS Method - Dimension: 5000, Time: 686.5791 seconds, Residual: 2846.7071, L2 Norm Difference: 69.9209
BFGS Method - Dimension: 10000, Time: 3962.6265 seconds, Residual: 3030.6079, L2 Norm Difference: 99.4245


MemoryError: Unable to allocate 2.98 GiB for an array with shape (20000, 20000) and data type float64